In [1]:
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional
import random
from numpy.random.mtrand import randint
import csv
from pathlib import Path
import json

In [2]:
rng = np.random.default_rng(seed=42)

In [3]:
INPUT_FILE = "bounded_polygons.geojson"


##Core demographic profiles
 Core demographic profiles defining statistical parameters for 9 socioeconomic groups
 Each profile includes means/standard deviations for key attributes and probability distributions
 Note: race_dist uses cumulative probabilities [White, Black, Asian, Other] for UK demographics


In [4]:
demographic_profiles = {
    'affluent_professional': {
        'age_mean': 42, 'age_std': 9,
        'education_mean': 6.5, 'education_std': 1.2,
        'income_mean': 65000, 'income_std': 20000,
        'race_dist': [0.85, 0.05, 0.08, 0.02],
        'family_ratio': 0.6,
        'urban_preference': 0.8,
        'rural_preference': 0.2,
        'household_type_probabilities': {
            'single_person': 0.15,       # Often live alone in city apartments
            'couple_no_children': 0.45,  # Very high - DINK lifestyle
            'nuclear_family': 0.30,      # Professional families
            'single_parent_family': 0.03, # Rare
            'shared_household': 0.02,    # Very rare
            'multi_generational': 0.05   # Some cultural variation
        },
        'typical_children': [0, 1, 2],    # If they have children
        'children_probabilities': [0.4, 0.4, 0.2],  # 0, 1, or 2 children
        'notes': 'High dual-income couples, late childbearing'
    },

    'middle_class': {
        'age_mean': 45, 'age_std': 10,
        'education_mean': 4.5, 'education_std': 1.5,
        'income_mean': 38000, 'income_std': 12000,
        'race_dist': [0.88, 0.04, 0.06, 0.02],
        'family_ratio': 0.7,
        'urban_preference': 0.6,
        'rural_preference': 0.4,
        'household_type_probabilities': {
            'single_person': 0.20,
            'couple_no_children': 0.30,   # Strong couple presence
            'nuclear_family': 0.35,       # Classic family unit
            'single_parent_family': 0.08,
            'shared_household': 0.04,
            'multi_generational': 0.03
        },
        'typical_children': [0, 1, 2, 3],
        'children_probabilities': [0.3, 0.4, 0.25, 0.05],
        'notes': 'Traditional family structures, suburban living'
    },

    'working_class': {
        'age_mean': 48, 'age_std': 11,
        'education_mean': 2.5, 'education_std': 1.2,
        'income_mean': 26000, 'income_std': 8000,
        'race_dist': [0.90, 0.03, 0.05, 0.02],
        'family_ratio': 0.65,
        'urban_preference': 0.4,
        'rural_preference': 0.6,
        'household_type_probabilities': {
            'single_person': 0.25,        # Often elderly or divorced
            'couple_no_children': 0.25,   # Traditional couples
            'nuclear_family': 0.30,       # Working-class families
            'single_parent_family': 0.15, # Higher than average
            'shared_household': 0.03,
            'multi_generational': 0.02
        },
        'typical_children': [1, 2, 3],
        'children_probabilities': [0.4, 0.4, 0.2],
        'notes': 'Earlier childbearing, larger families traditionally'
    },

    'student': {
        'age_mean': 22, 'age_std': 4,
        'education_mean': 5.5, 'education_std': 1.5,
        'income_mean': 12000, 'income_std': 8000,
        'race_dist': [0.75, 0.06, 0.15, 0.04],
        'family_ratio': 0.05,
        'urban_preference': 0.9,
        'rural_preference': 0.1,
        'household_type_probabilities': {
            'single_person': 0.10,        # In university accommodation
            'couple_no_children': 0.05,   # Student couples rare
            'nuclear_family': 0.01,       # Very rare (mature students)
            'single_parent_family': 0.01, # Very rare
            'shared_household': 0.80,     # VERY high - student houses
            'multi_generational': 0.03    # Living with parents
        },
        'typical_children': [0],          # Virtually no children
        'children_probabilities': [1.0],
        'notes': 'Mostly in shared housing, some in halls, few with families'
    },

    'retired': {
        'age_mean': 72, 'age_std': 8,
        'education_mean': 2.8, 'education_std': 1.5,
        'income_mean': 22000, 'income_std': 10000,
        'race_dist': [0.95, 0.02, 0.02, 0.01],
        'family_ratio': 0.3,
        'urban_preference': 0.3,
        'rural_preference': 0.7,
        'household_type_probabilities': {
            'single_person': 0.40,        # Very high - widows/widowers
            'couple_no_children': 0.45,   # Retired couples
            'nuclear_family': 0.02,       # Living with adult children
            'single_parent_family': 0.01,
            'shared_household': 0.01,     # Very rare
            'multi_generational': 0.11    # Living with family
        },
        'typical_children': [0],          # Children grown and left
        'children_probabilities': [1.0],
        'notes': 'High proportion living alone, many in care homes (not captured)'
    },

    'low_income': {
        'age_mean': 38, 'age_std': 12,
        'education_mean': 1.2, 'education_std': 1.0,
        'income_mean': 16000, 'income_std': 6000,
        'race_dist': [0.82, 0.08, 0.07, 0.03],
        'family_ratio': 0.5,
        'urban_preference': 0.7,
        'rural_preference': 0.3,
        'household_type_probabilities': {
            'single_person': 0.30,        # Often single in social housing
            'couple_no_children': 0.15,   # Less stable relationships
            'nuclear_family': 0.20,       # Families in poverty
            'single_parent_family': 0.25, # Very high
            'shared_household': 0.08,     # House sharing due to costs
            'multi_generational': 0.02
        },
        'typical_children': [1, 2, 3],
        'children_probabilities': [0.5, 0.3, 0.2],
        'notes': 'High single parenthood, housing insecurity'
    },

    'young_professional': {
        'age_mean': 28, 'age_std': 4,
        'education_mean': 6.0, 'education_std': 1.0,
        'income_mean': 32000, 'income_std': 10000,
        'race_dist': [0.80, 0.05, 0.12, 0.03],
        'family_ratio': 0.2,
        'urban_preference': 0.9,
        'rural_preference': 0.1,
        'household_type_probabilities': {
            'single_person': 0.35,        # City living
            'couple_no_children': 0.40,   # Dual-income couples
            'nuclear_family': 0.10,       # Early starters
            'single_parent_family': 0.02,
            'shared_household': 0.12,     # Flat shares in expensive cities
            'multi_generational': 0.01
        },
        'typical_children': [0, 1],       # Mostly no children
        'children_probabilities': [0.9, 0.1],
        'notes': 'Delayed childbearing, urban concentration'
    },

    'farmer_agricultural': {
        'age_mean': 52, 'age_std': 12,
        'education_mean': 2.0, 'education_std': 1.0,
        'income_mean': 28000, 'income_std': 15000,
        'race_dist': [0.98, 0.01, 0.005, 0.005],
        'family_ratio': 0.75,
        'urban_preference': 0.1,
        'rural_preference': 0.9,
        'household_type_probabilities': {
            'single_person': 0.15,        # Elderly farmers
            'couple_no_children': 0.25,   # Farming couples
            'nuclear_family': 0.40,       # Farm families
            'single_parent_family': 0.05,
            'shared_household': 0.02,     # Farm workers sometimes
            'multi_generational': 0.13    # Farm succession important
        },
        'typical_children': [1, 2, 3, 4], # Larger families traditionally
        'children_probabilities': [0.3, 0.4, 0.2, 0.1],
        'notes': 'Strong family farms, multi-generational working'
    },

    'rural_worker': {
        'age_mean': 46, 'age_std': 11,
        'education_mean': 1.8, 'education_std': 0.8,
        'income_mean': 22000, 'income_std': 7000,
        'race_dist': [0.96, 0.02, 0.01, 0.01],
        'family_ratio': 0.7,
        'urban_preference': 0.2,
        'rural_preference': 0.8,
        'household_type_probabilities': {
            'single_person': 0.20,
            'couple_no_children': 0.30,
            'nuclear_family': 0.35,       # Rural families
            'single_parent_family': 0.08,
            'shared_household': 0.04,     # Sometimes seasonal workers
            'multi_generational': 0.03
        },
        'typical_children': [1, 2, 3],
        'children_probabilities': [0.4, 0.4, 0.2],
        'notes': 'Similar to working class but more rural'
    }
}

##Geographic area
Geographic area type definitions showing demographic composition percentages. Each area type has a distinct socioeconomic mix — for example, university towns have high student concentrations (35%), while coastal areas are retirement-heavy (50% retired). Values represent proportional distributions that sum to approximately 1.0 within each area.

In [5]:
area_types = {
    'city': {  # Bristol, Leeds, Edinburgh, Cardiff
        'demographic_distribution': {
            'affluent_professional': 0.15,
            'young_professional': 0.15,
            'middle_class': 0.25,
            'student': 0.10,
            'working_class': 0.20,
            'low_income': 0.10,
            'retired': 0.05
        },
        'household_type_distribution': {
            'single_person': 0.30,
            'couple_no_children': 0.28,   # Strong couple presence
            'nuclear_family': 0.22,       # More families than major cities
            'single_parent_family': 0.08,
            'shared_household': 0.09,
            'multi_generational': 0.03
        },
        'urban': True,
        'population_density': 'high',
        'typical_household_size': 2.3
    },
    'agricultural': {  # East Anglia, Lincolnshire, parts of Yorkshire
        'demographic_distribution': {
            'farmer_agricultural': 0.25,
            'rural_worker': 0.20,
            'working_class': 0.20,
            'retired': 0.20,
            'middle_class': 0.10,
            'affluent_professional': 0.05
        },
        'household_type_distribution': {
            'single_person': 0.22,        # Elderly farmers, widows/widowers
            'couple_no_children': 0.20,   # Farming couples
            'nuclear_family': 0.35,       # STRONG - farm families with children
            'single_parent_family': 0.05, # Lower than urban
            'shared_household': 0.03,     # Very rare
            'multi_generational': 0.15    # HIGH - farm succession, elderly parents
        },
        'urban': False,
        'population_density': 'very_low',
        'typical_household_size': 3.1,    # Larger rural families
        'notes': 'Farm families often multi-generational'
    },
    'market_town': {  # Traditional rural hubs
        'demographic_distribution': {
            'middle_class': 0.30,
            'working_class': 0.25,
            'retired': 0.20,
            'rural_worker': 0.15,
            'affluent_professional': 0.10
        },
        'household_type_distribution': {
            'single_person': 0.25,
            'couple_no_children': 0.30,   # Strong - commuter couples
            'nuclear_family': 0.25,       # Good for families
            'single_parent_family': 0.08,
            'shared_household': 0.07,     # Some young workers
            'multi_generational': 0.05
        },
        'urban': False,
        'population_density': 'low_medium',
        'typical_household_size': 2.5,
        'commuter_patterns': 'Many commute to nearby cities'
    },
}

## Household Structure Definitions

Household structure type definitions showing composition patterns and size distributions. Each structure type represents a distinct living arrangement pattern — for example, single-parent households typically have 1-2 children (70% have 1 child), while multi-adult shared households often consist of 2-4 unrelated adults. Size distributions represent proportional likelihoods of different household sizes within each structure type, with values summing to 1.0 within each structure.

In [6]:
household_types = {
    # Key: (adults, children) where children = dependent children (<18)
    # Value: Probability distribution of household sizes within this structure

    # Single adult, no children
    'single_person': {
        'description': 'One adult living alone',
        'size_distribution': {
            1: 1.0  # Always exactly 1 person
        }
    },

    # Couple, no children
    'couple_no_children': {
        'description': 'Two adults in a relationship, no dependent children',
        'size_distribution': {
            2: 1.0  # Always exactly 2 people
        }
    },

    # Couple with children
    'nuclear_family': {
        'description': 'Two adults with 1+ dependent children',
        'size_distribution': {
            3: 0.40,  # 40% have 1 child
            4: 0.45,  # 45% have 2 children
            5: 0.12,  # 12% have 3 children
            6: 0.03   # 3% have 4+ children
        }
    },

    # Single parent with children
    'single_parent_family': {
        'description': 'One adult with 1+ dependent children',
        'size_distribution': {
            2: 0.70,  # 70% have 1 child
            3: 0.25,  # 25% have 2 children
            4: 0.05   # 5% have 3+ children
        }
    },

    # Multiple adults, no children (shared household)
    'shared_household': {
        'description': 'Multiple unrelated adults sharing accommodation',
        'size_distribution': {
            2: 0.40,  # 40% are 2 people
            3: 0.35,  # 35% are 3 people
            4: 0.20,  # 20% are 4 people
            5: 0.05   # 5% are 5+ people
        }
    },

    # Complex/multi-generational
    'multi_generational': {
        'description': 'Multiple generations or complex family structures',
        'size_distribution': {
            3: 0.30,  # 30% are 3 people
            4: 0.40,  # 40% are 4 people
            5: 0.20,  # 20% are 5 people
            6: 0.10   # 10% are 6+ people
        }
    }
}


##Core population generation
Core population generation function using "roulette wheel" selection. Creates individuals with demographic attributes drawn from statistical distributions. Implements proportional sampling to match area composition, generates unique IDs, and produces complete individual records ready for household formation. Returns list of person dictionaries with attributes like age, income, education, ethnicity, and family indicators.

In [7]:

def generate_individuals_wheel(area, area_id, population):
    """
    Generate synthetic individuals for a specific area using proportional selection.

    This function creates a population of individuals where:
    - Each individual belongs to a demographic profile (affluent, middle_class, etc.)
    - The proportion of each profile matches the area's composition
    - Individual attributes are drawn from normal distributions based on their profile

    Parameters:
    -----------
    area : dict
        Dictionary defining the demographic composition of the area
        Format: {'affluent': 0.3, 'middle_class': 0.4, 'working_class': 0.3}
        Keys are demographic profile names, values are proportions (should sum to ~1.0)

    area_id : str
        Unique identifier for this area (e.g., 'LON_01', 'MAN_02')
        Used to create unique individual IDs

    population : int
        Number of individuals to generate for this area

    Returns:
    --------
    list: List of individual dictionaries with demographic attributes
    """


    
    for i in range(population):
        wheel =[]
        keys = []
        count =0
        demographic = area["demographic_distribution"]
        for key,value in demographic.items():
            count+=value
            wheel.append(count)
            keys.append(key)

        indx=0    
        rnd_demographic= random.random*max(wheel)
        while rnd_demographic > wheel[indx]:
            count += 1

        # Step 2c: Get the selected demographic profile
        demographic_id = keys[indx]  # Profile name (e.g., 'middle_class')
        demographic = demographic_profiles[demographic_id]  # Full profile dictionary

        # Step 2d: Generate age from normal distribution
        # Clamp to reasonable bounds (implied - could add explicit bounds)
        age = int(rng.normal(
            loc=demographic["age_mean"],       # Center of distribution
            scale=demographic["age_std"],      # Spread of distribution
            size=1                             # Generate 1 value
        )[0])

        # Step 2e: Generate education level from normal distribution
        # UK system: 0-8 levels
        education = int(rng.normal(
            loc=demographic["education_mean"],
            scale=demographic["education_std"],
            size=1
        )[0])

        # Step 2f: Assign gender (0 or 1, typically 0=male, 1=female)
        # Equal probability for each gender (0.5 each)
        gender = random.randint(0, 1)

        # Step 2g: Assign ethnicity using another roulette wheel
        # race_dist is cumulative probabilities for ethnicities
        # Example: [0.7, 0.85, 0.95, 1.0] means:
        #   White: 70%, Black: 15%, Asian: 10%, Other: 5%
        wheel =[]
        keys = []
        count =0
        ethnic_dict = demographic["race_dist"]
        for key,value in ethnic_dict.items():
            count+=value
            wheel.append(count)
            keys.append(key)

        rnd_race = random.random() * max(wheel)  # Scale to last cumulative value

        # Find which ethnicity corresponds to the random number
        indx = 0
        while race_rnd > wheel[indx]:
            indx += 1
        ethnicity = indx  # 0=White, 1=Black, 2=Asian, 3=Other

        # Step 2h: Get family ratio (probability of being in a couple/family)
        family_ratio = demographic["family_ratio"]

        # Step 2i: Generate income from normal distribution (£)
        income = int(rng.normal(
            loc=demographic["income_mean"],
            scale=demographic["income_std"],
            size=1
        )[0])

        # Step 2j: Create individual dictionary with all attributes
        individual = {
            "id": str(area_id) + "_" + str(i),          # Unique ID: "LON_01_0", "LON_01_1", etc.
            "demographic_id": demographic_id,           # Profile type
            "age": age,                                 # Integer age
            "education": education,                     # Education level (0-8)
            "gender": gender,                           # 0 or 1
            "ethnicity": ethnicity,                     # 0, 1, 2, or 3
            "family_ratio": family_ratio,               # Probability of being in couple
            "income": income,                           # Annual income in £
            "in_household": False,                      # Tracking for household formation
            "household_id": "",                         # Will be filled during household creation
            "relationship": ""                          # Will be "head", "partner", etc.
        }

        # Add individual to this area's population
        local_population.append(individual)

    # Step 3: Return complete population for this area
    return local_population


In [8]:

def generate_individuals(area, area_id,admin_area, population):
    """
    Generate synthetic individuals for a specific area using proportional selection.

    This function creates a population of individuals where:
    - Each individual belongs to a demographic profile (affluent, middle_class, etc.)
    - The proportion of each profile matches the area's composition
    - Individual attributes are drawn from normal distributions based on their profile

    Parameters:
    -----------
    area : dict
        Dictionary defining the demographic composition of the area
        Format: {'affluent': 0.3, 'middle_class': 0.4, 'working_class': 0.3}
        Keys are demographic profile names, values are proportions (should sum to ~1.0)

    area_id : str
        Unique identifier for this area (e.g., 'LON_01', 'MAN_02')
        Used to create unique individual IDs

    population : int
        Number of individuals to generate for this area

    Returns:
    --------
    list: List of individual dictionaries with demographic attributes
    """

    # Step 1: Create a "roulette wheel" for proportional selection
    # This allows us to randomly select demographic profiles according to area proportions
    wheel = []        # Cumulative probability wheel
    local_population = []  # List to store generated individuals
    summation = 0     # Running total of probabilities (should end at ~1.0)

    # Build the cumulative probability wheel
    for key, value in area["demographic_distribution"].items():
        # Add current profile's proportion to running total
        summation += value

        # Store cumulative probability and profile name
        # Example: If area = {'affluent': 0.3, 'middle_class': 0.4, 'working_class': 0.3}
        # wheel becomes: [[0.3, 'affluent'], [0.7, 'middle_class'], [1.0, 'working_class']]
        wheel.append([summation, key])

    # Step 2: Generate the specified number of individuals
    for i in range(population):
        # Step 2a: Spin the roulette wheel - pick a random number 0 to total probability
        rnd = random.random() * summation

        # Step 2b: Find which profile this random number corresponds to
        count = 0
        # Linear search through cumulative probabilities
        # Example: rnd = 0.5, wheel = [[0.3, 'affluent'], [0.7, 'middle_class'], [1.0, 'working_class']]
        # 0.5 > 0.3? Yes → count=1
        # 0.5 > 0.7? No → stop, demographic_id = 'middle_class'
        while rnd > wheel[count][0]:
            count += 1

        # Step 2c: Get the selected demographic profile
        demographic_id = wheel[count][1]  # Profile name (e.g., 'middle_class')
        demographic = demographic_profiles[demographic_id]  # Full profile dictionary

        # Step 2d: Generate age from normal distribution
        # Clamp to reasonable bounds (implied - could add explicit bounds)
        age = int(rng.normal(
            loc=demographic["age_mean"],       # Center of distribution
            scale=demographic["age_std"],      # Spread of distribution
            size=1                             # Generate 1 value
        )[0])

        # Step 2e: Generate education level from normal distribution
        # UK system: 0-8 levels
        education = int(rng.normal(
            loc=demographic["education_mean"],
            scale=demographic["education_std"],
            size=1
        )[0])

        # Step 2f: Assign gender (0 or 1, typically 0=male, 1=female)
        # Equal probability for each gender (0.5 each)
        gender = random.randint(0, 1)

        # Step 2g: Assign ethnicity using another roulette wheel
        # race_dist is cumulative probabilities for ethnicities
        # Example: [0.7, 0.85, 0.95, 1.0] means:
        #   White: 70%, Black: 15%, Asian: 10%, Other: 5%
        race_rnd = random.random() * demographic["race_dist"][-1]  # Scale to last cumulative value

        # Find which ethnicity corresponds to the random number
        count = 0
        while race_rnd > demographic["race_dist"][count]:
            count += 1
        ethnicity = count  # 0=White, 1=Black, 2=Asian, 3=Other

        # Step 2h: Get family ratio (probability of being in a couple/family)
        family_ratio = demographic["family_ratio"]

        # Step 2i: Generate income from normal distribution (£)
        income = int(rng.normal(
            loc=demographic["income_mean"],
            scale=demographic["income_std"],
            size=1
        )[0])

        # Step 2j: Create individual dictionary with all attributes
        individual = {
            "area": str(area_id),
            "admin_area" : str(admin_area),
            "id": str(area_id) + "_" + str(i),          # Unique ID: "LON_01_0", "LON_01_1", etc.
            "demographic_id": demographic_id,           # Profile type
            "age": age,                                 # Integer age
            "education": education,                     # Education level (0-8)
            "gender": gender,                           # 0 or 1
            "ethnicity": ethnicity,                     # 0, 1, 2, or 3
            "family_ratio": family_ratio,               # Probability of being in couple
            "income": income,                           # Annual income in £
            "in_household": False,                      # Tracking for household formation
            "household_id": "",                         # Will be filled during household creation
            "relationship": ""                          # Will be "head", "partner", etc.
        }

        # Add individual to this area's population
        local_population.append(individual)

    # Step 3: Return complete population for this area
    return local_population


##Probability Combination Function

The combine_probabilities() function implements a simple Bayesian approach to household type determination by multiplying individual and area probabilities. This method follows the logical principle that a household type should be considered likely only if it's plausible both for the individual's demographic profile and for the geographic area type. After multiplication, probabilities are renormalized to ensure they sum to 1.0, creating a valid probability distribution ready for weighted random selection via roulette wheel sampling. The fallback to equal probability distribution ensures robustness when all combined probabilities evaluate to zero.

In [9]:
def combine_probabilities(person_probs, area_probs):
    """
    EVEN SIMPLER: Just multiply
    """
    combined = {}

    for hh_type in set(person_probs.keys()) | set(area_probs.keys()):
        p = person_probs.get(hh_type, 0)
        a = area_probs.get(hh_type, 0)
        combined[hh_type] = p * a

    # Renormalize
    total = sum(combined.values())
    if total > 0:
        combined = {k: v/total for k, v in combined.items()}
    else:
        # Fallback to equal probability
        combined = {k: 1/len(combined) for k in combined.keys()}
    hh_type =[]
    wheel=[]
    count=0
    for key, value in combined.items():
        hh_type.append(key)
        count+=value
        wheel.append(count)

    return hh_type,wheel

##Relationship compatibility
Relationship compatibility scoring algorithm. Computes similarity between two individuals based on multiple weighted factors: age, education, income (log-scaled), ethnicity, and demographic profile. Includes hard filters for gender preference (90% heterosexual assumption) and family willingness. Returns 0-1 score where higher values indicate better match potential for household formation.

In [10]:
def calculate_similarity_score_non_gender(person1, person2):

    # 1. Age similarity: Closer age = higher score
    age_diff = abs(person1['age'] - person2['age'])
    age_similarity = 1 / (1 + age_diff / 5)  # Normalized, 5-year difference = 0.5

    # 2. Education similarity: Similar education level = higher score
    edu_diff = abs(person1['education'] - person2['education'])
    edu_similarity = 1 / (1 + edu_diff / 2)  # Normalized, 2-level difference = 0.5

    # 3. Income similarity (log scale, as income differences matter less at high incomes)
    income1 = max(person1['income'], 10000)  # Avoid log(0)
    income2 = max(person2['income'], 10000)
    income_ratio = max(income1, income2) / min(income1, income2)
    income_similarity = 1 / (1 + np.log(income_ratio))  # Log scale normalization

    # 4. Ethnicity similarity: Same ethnicity
    ethnicity_similarity = 1.0 if person1['ethnicity'] == person2['ethnicity'] else 0.3

    # 5. Profile similarity: Same demographic profile = bonus
    same_profile = person1['demographic_id'] == person2['demographic_id']

    # Combine all components with weights
    weights = {
        'age': 0.25,        # Age is important but not everything
        'education': 0.20,  # Education level matters for compatibility
        'income': 0.20,     # Similar socioeconomic status
        'ethnicity': 0.15,  # Ethnic homophily (people tend to partner within ethnicity)
        'profile': 0.20     # Overall demographic profile similarity
    }

    # Calculate weighted sum
    score = (
        weights['age'] * age_similarity +
        weights['education'] * edu_similarity +
        weights['income'] * income_similarity +
        weights['ethnicity'] * ethnicity_similarity
    )
    return score

##Couple Compatibility

The calculate_similarity_score() function implements a partner compatibility algorithm that evaluates potential romantic/cohabiting partnerships between two adult individuals. It combines demographic similarity assessment with realistic social constraints to determine relationship viability.

In [11]:
def calculate_similarity_score(person1, person2):
    if person1['gender'] == person2['gender'] and random.random() > 0.9:  # 90% heterosexual
        return 0.0

    # 1. Age similarity: Closer age = higher score
    age_diff = abs(person1['age'] - person2['age'])
    age_similarity = 1 / (1 + age_diff / 5)  # Normalized, 5-year difference = 0.5

    # 2. Education similarity: Similar education level = higher score
    edu_diff = abs(person1['education'] - person2['education'])
    edu_similarity = 1 / (1 + edu_diff / 2)  # Normalized, 2-level difference = 0.5

    # 3. Income similarity (log scale, as income differences matter less at high incomes)
    income1 = max(person1['income'], 10000)  # Avoid log(0)
    income2 = max(person2['income'], 10000)
    income_ratio = max(income1, income2) / min(income1, income2)
    income_similarity = 1 / (1 + np.log(income_ratio))  # Log scale normalization

    # 4. Ethnicity similarity: Same ethnicity
    ethnicity_similarity = 1.0 if person1['ethnicity'] == person2['ethnicity'] else 0.3

    # 5. Profile similarity: Same demographic profile = bonus
    same_profile = person1['demographic_id'] == person2['demographic_id']

    # Combine all components with weights
    weights = {
        'age': 0.25,        # Age is important but not everything
        'education': 0.20,  # Education level matters for compatibility
        'income': 0.20,     # Similar socioeconomic status
        'ethnicity': 0.15,  # Ethnic homophily (people tend to partner within ethnicity)
        'profile': 0.20     # Overall demographic profile similarity
    }

    # Calculate weighted sum
    score = (
        weights['age'] * age_similarity +
        weights['education'] * edu_similarity +
        weights['income'] * income_similarity +
        weights['ethnicity'] * ethnicity_similarity
    )
    return score

##Multi Generational
The multi_generational() function implements compatibility algorithm that evaluates potential romantic/cohabiting partnerships between two adult individuals and then adds additional older houshold members. It combines demographic similarity assessment with realistic social constraints to determine relationship viability.

In [12]:
def multi_generational(pop,processed,household,hh_size,person1,hh_id):
    potential = []
    for p in pop:
        # Compute similarity (e.g., age, interests, socioeconomic status).
        similarity = calculate_similarity_score(person1, p)
        # Only keep candidates that are sufficiently similar.
        if similarity > 0.3:
            potential.append(p)

    # If we found at least one candidate, pick one at random.
    if potential:
        person2 = random.choice(potential)
        # Remove the chosen partner from the pool and record them.
        if person2 in pop:
            pop.remove(person2)
            processed.append(person2)

            # Update partner metadata.
            person2["in_household"] = True
            person2["relationship"] = "partner"
            person2["household_id"] = hh_id

            # Add the partner to the household’s member list.
            household["members"].append(person2)

    potential = []
    for p in pop:
        # Only keep candidates that of correct age.
        if p["age"]>person1["age"]+16:
            potential.append(p)
    if hh_size>2:
        hh_size=hh_size-2
        for i in range(hh_size):
            if potential:
                person2 = random.choice(potential)
                # Remove the chosen partner from the pool and record them.
                if person2 in pop:
                    pop.remove(person2)
                    processed.append(person2)

                    # Update partner metadata.
                    person2["in_household"] = True
                    person2["relationship"] = "partner"
                    person2["household_id"] = hh_id

                    # Add the partner to the household’s member list.
                    household["members"].append(person2)

##Shared_Household
The shared_household() function implements compatibility algorithm that evaluates potential simmilar members of a shared household


In [13]:
def shared_household(pop,processed,household,hh_size,person1,hh_id):
    potential = []
    for p in pop:
        # Compute similarity (e.g., age, interests, socioeconomic status).
        similarity = calculate_similarity_score_non_gender (person1, p)
        # Only keep candidates that are sufficiently similar.
        if similarity > 0.3:
            potential.append(p)
    for i in range(hh_size):
        if potential:
            person2 = random.choice(potential)
            # Remove the chosen partner from the pool and record them.
            if person2 in pop:
                pop.remove(person2)
                processed.append(person2)

                # Update partner metadata.
                person2["in_household"] = True
                person2["relationship"] = "partner"
                person2["household_id"] = hh_id

                # Add the partner to the household’s member list.
                household["members"].append(person2)

##Couple
couple() Calls the calculate_similarity_score for all families




In [14]:
def couple(pop,processed,household,hh_size,person1,hh_id):
    potential = []
    for p in pop:
        # Compute similarity (e.g., age, interests, socioeconomic status).
        similarity = calculate_similarity_score(person1, p)
        # Only keep candidates that are sufficiently similar.
        if similarity > 0.3:
            potential.append(p)

    # If we found at least one candidate, pick one at random.
    if potential:
        person2 = random.choice(potential)
        # Remove the chosen partner from the pool and record them.
        if person2 in pop:
            pop.remove(person2)
            processed.append(person2)

            # Update partner metadata.
            person2["in_household"] = True
            person2["relationship"] = "partner"
            person2["household_id"] = hh_id

            # Add the partner to the household’s member list.
            household["members"].append(person2)

##Household formation
Main household formation pipeline. Generates individuals for an area, then organizes them into households through sequential assignment. Each household starts with a randomly selected "head," then optionally adds a compatible partner based on similarity scoring and family ratio probability. Returns two complementary views: a flat list of all individuals with household assignments, and a structured collection of household groups with metadata. Creates realistic family units while preserving demographic distributions.

In [15]:


def create_area_population(area_type, urban,area_id,admin_area, population):
    """
    Generate a list of individuals for a given area and then group them
    into households.

    Parameters
    ----------
    area_type : str
        The type/category of the area (e.g., “city”, “rural”).
    area_id   : str
        Unique identifier for the area – also used as a prefix for household IDs.
    urban     : bool
        Flag indicating whether the area is urban (True) or rural (False).
    population: int
        Desired number of individuals to generate for the area.

    Returns
    -------
    processed   : list
        All individuals after they have been assigned to households.
    households  : list
        A collection of household dictionaries, each containing its members.
    """

    # ----------------------------------------------------------------------
    # 1 GENERATE BASE POPULATION
    # ----------------------------------------------------------------------
    # Create initial pool of individuals with demographic attributes
    # generate_individuals() uses roulette wheel selection to ensure
    # demographic mix matches area_type["demographic_distribution"]
    pop = generate_individuals(area_type, area_id,admin_area, population)

    # Containers for final results
    processed = []          # All individuals after household assignment
    households = []         # Structured household data
    households_count = 0    # Counter for unique household IDs

    # ----------------------------------------------------------------------
    # 2 HOUSEHOLD FORMATION LOOP
    # ----------------------------------------------------------------------
    # Continue until all individuals are placed in households
    while len(pop) > 0:
        # Create unique household identifier
        hh_id = f"{area_id}_hh{households_count}"

        # --------------------------------------------------------------
        # 2a SELECT HOUSEHOLD HEAD AND DETERMINE HOUSEHOLD TYPE
        # --------------------------------------------------------------
        # Randomly select an individual to become household head
        person1 = random.choice(pop)

        # Get area-specific household type probabilities
        area_household_probs = area_type["household_type_distribution"]

        # Get individual's demographic profile
        demographic = person1["demographic_id"]

        # --------------------------------------------------------------
        # PROBABILITY-BLENDING FOR HOUSEHOLD TYPE SELECTION
        # --------------------------------------------------------------
        # Retrieve individual's demographic profile household probabilities
        person_household_probs = demographic_profiles[demographic]["household_type_probabilities"]

        # Combine individual and area probabilities using Bayesian multiplication
        # combine_probabilities() returns:
        # - hh_types: list of household type keys
        # - hh_probabilities: cumulative probabilities for roulette wheel selection
        hh_types, hh_probabilities = combine_probabilities(
            person_household_probs,
            area_household_probs
        )

        # Roulette wheel selection for household type
        hh_prob = random.random() * max(hh_probabilities)
        indx = 0
        while hh_prob > hh_probabilities[indx]:
            indx += 1
        hh_type_key = hh_types[indx]  # Selected household type (e.g., "nuclear_family")

        # Get full household type definition
        hh_type = household_types[hh_type_key]

        # --------------------------------------------------------------
        # DETERMINE HOUSEHOLD SIZE
        # --------------------------------------------------------------
        # Use household type's size distribution to determine number of occupants
        size_distribution = hh_type["size_distribution"]

        # Build roulette wheel for size selection
        wheel = []
        sizes = []
        count = 0
        for key, value in size_distribution.items():
            count += value
            sizes.append(key)       # Possible sizes (e.g., [1, 2, 3, 4])
            wheel.append(count)     # Cumulative probabilities

        # Roulette wheel selection for household size
        occupants_rnd = random.random() * max(wheel)
        indx = 0
        while occupants_rnd > wheel[indx]:
            indx += 1
        hh_size = sizes[indx]  # Final household size

        # --------------------------------------------------------------
        # 2b INITIALIZE HOUSEHOLD WITH HEAD
        # --------------------------------------------------------------
        # Update head's metadata
        person1["in_household"] = True
        person1["relationship"] = "head"
        person1["household_id"] = hh_id

        # Move head from pool to processed list
        processed.append(person1)
        pop.remove(person1)

        # Create household dictionary with head as first member
        household = {
            "area_id":area_id,
            "admin_area":admin_area,
            "hh_id": hh_id,
            "urban": urban,
            "members": [person1],  # Start with head only
        }

        # --------------------------------------------------------------
        # 2c ADD ADDITIONAL HOUSEHOLD MEMBERS BASED ON TYPE
        # --------------------------------------------------------------
        # Note: hh_size includes the head, so we need to add (hh_size - 1) members

        if hh_type_key in ["nuclear_family", "couple_no_children"]:
            # COUPLE-BASED HOUSEHOLDS: Find compatible romantic partner
            # couple() adds 1 partner (if available) based on similarity scoring
            couple(pop, processed, household, hh_size, person1, hh_id)

        elif hh_type_key == "multi_generational":
            # MULTI-GENERATIONAL: Add partner first, then older household members
            # multi_generational() adds partner + (hh_size-2) older members
            multi_generational(pop, processed, household, hh_size, person1, hh_id)

        elif hh_type_key == "shared_household":
            # SHARED HOUSEHOLD: Add compatible non-romantic housemates
            # shared_household() adds (hh_size-1) members based on similarity
            shared_household(pop, processed, household, hh_size, person1, hh_id)

        # Note: "single_person" households require no additional members

        # --------------------------------------------------------------
        # 2d FINALIZE AND STORE HOUSEHOLD
        # --------------------------------------------------------------
        # Record actual household size (may differ from target if insufficient candidates)
        household["size"] = len(household["members"])

        # Add to households collection
        households.append(household)
        households_count += 1

    # ----------------------------------------------------------------------
    # 3 RETURN RESULTS
    # ----------------------------------------------------------------------
    return processed, households


In [16]:
def calculate_census(households,area_id):
    """
    Compute a set of summary statistics for a collection of households.

    Parameters
    ----------
    households : list[dict]
        Each household dictionary must contain:
          - "members": a list of member dictionaries
          - "urban":   a boolean (True if the household is urban)

        Each member dictionary must contain:
          - "gender": 0 for male, 1 for female
          - "age":    integer age in years
    Returns
    -------
    dict
        Aggregated counts such as total population, number of urban/rural
        households, size‑distribution of households, and age‑by‑gender bins.
    """

    # ------------------------------------------------------------------
    # Initialise all counters to zero
    # ------------------------------------------------------------------
    population       = 0   # total number of individuals
    hh_population    = 0   # total number of households processed

    location_urban   = 0   # count of urban households
    location_rural   = 0   # count of rural households

    # Household‑size distribution (1‑person, 2‑person, …)
    hh_size1 = 0
    hh_size2 = 0
    hh_size3 = 0
    hh_size4 = 0   # “4 or more” bucket

    # Age‑by‑gender buckets (male/female, three age ranges)
    m_0_to_30   = 0
    f_0_to_30   = 0
    m_30_to_60  = 0
    f_30_to_60  = 0
    m_gt_60     = 0
    f_gt_60     = 0
    
    # Education
    edu_non     = 0 
    edu_low     = 0 
    edu_middle  = 0 
    edu_high    = 0
    # ------------------------------------------------------------------
    # Iterate over every household in the input list
    # ------------------------------------------------------------------
    for h in households:
        # --------------------------------------------------------------
        # Household‑level calculations
        # --------------------------------------------------------------
        hh_size = len(h["members"])   # how many people live here?
        hh_population += 1            # we’ve seen another household
        population += hh_size         # add its members to the total pop

        # Urban vs. rural classification
        if h["urban"]:
            location_urban += 1
        else:
            location_rural += 1

        # Update the household‑size histogram
        if hh_size == 1:
            hh_size1 += 1
        elif hh_size == 2:
            hh_size2 += 1
        elif hh_size == 3:
            hh_size3 += 1
        else:
            hh_size4 += hh_size

        # --------------------------------------------------------------
        # Member‑level calculations (age & gender breakdown)
        # --------------------------------------------------------------
        for m in h["members"]:
            gender = m["gender"]   # 0 = male, 1 = female
            age    = m["age"]
            edu    = m["education"]

            # Male branch
            if gender == 0:
                if age < 30:
                    m_0_to_30 += 1
                elif age < 60:
                    m_30_to_60 += 1
                else:
                    m_gt_60 += 1
            # Female branch
            else:
                if age < 30:
                    f_0_to_30 += 1
                elif age < 60:
                    f_30_to_60 += 1
                else:
                    f_gt_60 += 1
            
            #Education
            if edu == 0:
                edu_non+=1
            elif edu <=2:
                edu_low+=1 
            elif edu <=6:
                edu_middle+=1 
            else:
                edu_high+=1
                

    # ------------------------------------------------------------------
    # Pack everything into a dictionary for easy downstream consumption
    # ------------------------------------------------------------------
    return {
        "area_id":         area_id,
        "population":      len(households),
        "location_urban":  location_urban,
        "location_rural":  location_rural,
        "hh_size1":        hh_size1,
        "hh_size2":        hh_size2,
        "hh_size3":        hh_size3,
        "hh_size4":        hh_size4,
        "m_0_to_30":       m_0_to_30,
        "f_0_to_30":       f_0_to_30,
        "m_30_to_60":      m_30_to_60,
        "f_30_to_60":      f_30_to_60,
        "m_gt_60":         m_gt_60,
        "f_gt_60":         f_gt_60,
        "edu_non":         edu_non,
        "edu_low":         edu_low, 
        "edu_middle":      edu_middle,
        "edu_high":        edu_high
    }

In [17]:
def calculate_survey(household):
    """
    Produce summary statistics for a *single* household that came from a survey.

    Parameters
    ----------
    household : dict
        Expected keys:
            - "hh_id"   : unique identifier for the household
            - "urban"   : bool  True if the household is urban, False otherwise
            - "members" : list of member dicts, each with:
                  * "gender" (0 = male, 1 = female)
                  * "age"    (integer)

    Returns
    -------
    tuple(dict, list)
        1 A dictionary containing aggregated counts for this household.
        2 The original ``members`` list (so the caller still has the raw data).
    """

    # ------------------------------------------------------------------
    # Initialise all counters to zero
    # ------------------------------------------------------------------
    location_urban = 0
    location_rural = 0

    # Household‑size distribution (1‑person, 2‑person, 3‑person, 4+)
    hh_size1 = 0
    hh_size2 = 0
    hh_size3 = 0
    hh_size4 = 0   # “4 or more” bucket

    # Age‑by‑gender buckets
    m_0_to_30 = 0
    f_0_to_30 = 0
    m_30_to_60 = 0
    f_30_to_60 = 0
    m_gt_60 = 0
    f_gt_60 = 0

    # Education
    edu_non     = 0 
    edu_low     = 0 
    edu_middle  = 0 
    edu_high    = 0

    # ------------------------------------------------------------------
    # Extract basic household metadata
    # ------------------------------------------------------------------
    hh_id   = household["hh_id"]
    hh_size = len(household["members"])

    # Urban vs. rural classification
    if household["urban"]:
        location_urban += 1
    else:
        location_rural += 1

    # Update the household‑size histogram
    if hh_size == 1:
        hh_size1 += 1
    elif hh_size == 2:
        hh_size2 += 1
    elif hh_size == 3:
        hh_size3 += 1
    else:
        hh_size4 += hh_size

    # ------------------------------------------------------------------
    # Loop over each member to fill the gender/age buckets
    # ------------------------------------------------------------------
    for m in household["members"]:
        gender = m["gender"]   # 0 = male, 1 = female
        age    = m["age"]
        edu    = m["education"]

        if gender == 0:        # ----- Male -----
            if age < 30:
                m_0_to_30 += 1
            elif age < 60:
                m_30_to_60 += 1
            else:
                m_gt_60 += 1
        else:                  # ----- Female -----
            if age < 30:
                f_0_to_30 += 1
            elif age < 60:
                f_30_to_60 += 1
            else:
                f_gt_60 += 1

        #Education
        if edu == 0:
            edu_non+=1
        elif edu <=2:
            edu_low+=1 
        elif edu <=6:
            edu_middle+=1 
        else:
            edu_high+=1

    # ------------------------------------------------------------------
    # Assemble the result dictionary
    # ------------------------------------------------------------------
    summary = {
        "hh_id":          hh_id,
        "location_urban": location_urban,
        "location_rural": location_rural,
        "hh_size1":       hh_size1,
        "hh_size2":       hh_size2,
        "hh_size3":       hh_size3,
        "hh_size4":       hh_size4,
        "m_0_to_30":      m_0_to_30,
        "f_0_to_30":      f_0_to_30,
        "m_30_to_60":     m_30_to_60,
        "f_30_to_60":     f_30_to_60,
        "m_gt_60":        m_gt_60,
        "f_gt_60":        f_gt_60,
        "edu_non":         edu_non,
        "edu_low":         edu_low, 
        "edu_middle":      edu_middle,
        "edu_high":        edu_high
    }

    # Return both the aggregated stats and the raw member list
    return summary, household["members"]


In [18]:
### Note this may take a few seconds

In [19]:
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    geojson_obj = json.load(f)
population = []
households=[]
census =[]
count = 0;
for t in geojson_obj["features"]:
    area_type =t["properties"]["type"]
    urban = t["properties"]["urban"]
    area_id = "d"+str(t["properties"]["id"])
    admin_area ="a"+str(t["properties"]["admin_area"])
    local_population, local_households = create_area_population(area_types[area_type],urban,area_id,admin_area,t["properties"]["population"])
    house_hold_census = calculate_census(local_households,area_id)
    census.append(house_hold_census)
    population= population+local_population
    households=households+local_households
    count+=1
    if count % 10 == 0:
        print(f"{count} Areas created")

10 Areas created
20 Areas created
30 Areas created
40 Areas created
50 Areas created
60 Areas created
70 Areas created
80 Areas created
90 Areas created
100 Areas created
110 Areas created
120 Areas created
130 Areas created
140 Areas created
150 Areas created
160 Areas created
170 Areas created
180 Areas created
190 Areas created
200 Areas created
210 Areas created


In [20]:
print("done")

done


In [21]:
def dicts_to_csv(dict_list: list[dict], outfile: str | Path) -> None:
    """
    Write *dict_list* to *outfile* as a CSV.

    • All dictionaries should share the same set of keys – those keys are used
      as the header row.
    • Missing keys in a particular row are written as empty cells.
    • The function overwrites *outfile* if it already exists.
    """
    if not dict_list:
        raise ValueError("The input list is empty – nothing to write.")

    # Preserve the order of keys as they appear in the first dict.
    # (Python 3.7+ guarantees dict order is insertion order.)
    fieldnames = list(dict_list[0].keys())

    # Ensure every dict has the same keys – fill missing ones with None.
    normalized = [
        {k: d.get(k, None) for k in fieldnames} for d in dict_list
    ]

    with open(outfile, mode="w", newline="", encoding="utf-8") as fp:
        writer = csv.DictWriter(fp, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(normalized)

In [22]:
dicts_to_csv(census, "artifical_cencus.csv")

In [23]:
dicts_to_csv(population, "artifical_population.csv")

In [24]:
survay = random.sample(households, int(len(households)/100))

In [25]:
len(survay)

593

In [26]:
hh_survay = []
full_ind_survey = []
for hh in survay:
    hh_data,members =calculate_survey(hh)
    full_ind_survey = full_ind_survey + members
    hh_survay.append(hh_data)

In [27]:
dicts_to_csv(hh_survay, "artifical_survay.csv")


In [28]:
dicts_to_csv(full_ind_survey, "compleate_artifical_individual_survey.csv")
